Dataset airbnb

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns


ruta = r"airbnb-listings-extract.csv"

df = pd.read_csv(ruta, sep=None, engine='python')
df.head()

Filas y columnas y formas del dataset

In [ ]:
print("Filas y columnas:", df.shape)
df.describe(include='all')
df.dtypes

Seleccionar la columnas y limpiar

In [ ]:
columnas_utiles = [
    "Accommodates", "Bathrooms", "Bedrooms", "Beds",
    "Latitude", "Longitude",
    "Price", "Cleaning Fee", "Security Deposit",
    "Guests Included", "Minimum Nights", "Maximum Nights",
    "Room Type", "Property Type"
]

df2 = df[columnas_utiles].copy()


def limpiar_precio(x):
    if isinstance(x, str):
        x = x.replace("$", "").replace(",", "")
    return pd.to_numeric(x, errors="coerce")

df2["Price"] = df2["Price"].apply(limpiar_precio)
df2["Cleaning Fee"] = df2["Cleaning Fee"].apply(limpiar_precio)
df2["Security Deposit"] = df2["Security Deposit"].apply(limpiar_precio)


df2 = df2[df2["Price"].notnull()]
df2 = df2[df2["Price"] > 0]
df2.isnull().mean().sort_values(ascending=False)

Grafico heatmap

Se puede observar la relacion entre las variables del dataset especialmente relacionados con el precio donde bedrooms, beds, cleaning fee, accomodates y guests included tienen correlaciones mayores a 0.50 esto se puede interpretar como que los lugares mas espaciosos suelen tener los precios mas altos.

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df2.corr(numeric_only=True), annot=True, cmap="Blues")
plt.title("Correlación entre variables numéricas")
plt.show()

Grafico histograma

En los histogramas se puede ver que la mayoria de los graficos estan sesgados a la izquierda lo que quiere decir que la mayoria de alojamientos tienen valores bajos con pocas habitaciones, baja capacidad, pocas camas, tarifas bajas y los valores altos son menos frecuentes en airbnb.

In [ ]:
df2.hist(figsize=(12,10), bins=20)
plt.tight_layout()
plt.show()

Datos preparados

In [7]:
y = df2["Price"]
X = df2.drop("Price", axis=1)
X = X.dropna()
y = y.loc[X.index]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


columnas_categoricas = ["Room Type", "Property Type"]
columnas_numericas = [c for c in X.columns if c not in columnas_categoricas]

preprocesamiento = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), columnas_categoricas),
        ("num", "passthrough", columnas_numericas)
    ]
)

Regresion lineal modelo

In [ ]:
modelo_lr = Pipeline(steps=[
    ("prep", preprocesamiento),
    ("lr", LinearRegression())
])

modelo_lr.fit(X_train, y_train)

pred_lr = modelo_lr.predict(X_test)
rmse_lr = np.sqrt(mean_squared_error(y_test, pred_lr))

rmse_lr

Random forest

In [ ]:
modelo_rf = Pipeline(steps=[
    ("prep", preprocesamiento),
    ("rf", RandomForestRegressor(n_estimators=200, random_state=42))
])

modelo_rf.fit(X_train, y_train)

pred_rf = modelo_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, pred_rf))


rmse_rf

Comparacion de metodos

In [ ]:
print("RMSE Regresión Lineal:", rmse_lr)
print("RMSE Random Forest:", rmse_rf)
